In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 3, Finished, Available)

In [8]:
path = 'Files/bronze/raye_tracks.json'
df = spark.read.json(path)

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 10, Finished, Available)

In [9]:
df.schema

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 11, Finished, Available)

StructType([StructField('tracks', ArrayType(StructType([StructField('album', StructType([StructField('album_type', StringType(), True), StructField('artists', ArrayType(StructType([StructField('external_urls', StructType([StructField('spotify', StringType(), True)]), True), StructField('href', StringType(), True), StructField('id', StringType(), True), StructField('name', StringType(), True), StructField('type', StringType(), True), StructField('uri', StringType(), True)]), True), True), StructField('external_urls', StructType([StructField('spotify', StringType(), True)]), True), StructField('href', StringType(), True), StructField('id', StringType(), True), StructField('images', ArrayType(StructType([StructField('height', LongType(), True), StructField('url', StringType(), True), StructField('width', LongType(), True)]), True), True), StructField('is_playable', BooleanType(), True), StructField('name', StringType(), True), StructField('release_date', StringType(), True), StructField('

In [10]:
#Flatten array of structs and structs
def flatten(df):
   # compute Complex Fields (Lists and Structs) in Schema   
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
    
      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)
    
      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):    
         df=df.withColumn(col_name,explode_outer(col_name))
    
      # recompute remaining Complex Fields in Schema       
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 12, Finished, Available)

In [11]:
df_flatten = flatten(df)
display(df_flatten)

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 13, Finished, Available)

Processing :tracks Type : <class 'pyspark.sql.types.ArrayType'>
Processing :tracks Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_album Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_artists Type : <class 'pyspark.sql.types.ArrayType'>
Processing :tracks_artists Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_external_ids Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_external_urls Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_album_artists Type : <class 'pyspark.sql.types.ArrayType'>
Processing :tracks_album_artists Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_album_external_urls Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_album_images Type : <class 'pyspark.sql.types.ArrayType'>
Processing :tracks_album_images Type : <class 'pyspark.sql.types.StructType'>
Processing :tracks_artists_external_urls Type : <class 'pyspark.sql.types.StructType'>
Processing :tr

SynapseWidget(Synapse.DataFrame, 489fa28b-7200-45e1-b926-68f28f0db36b)

In [12]:
df_flatten.columns

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 14, Finished, Available)

['tracks_disc_number',
 'tracks_duration_ms',
 'tracks_explicit',
 'tracks_href',
 'tracks_id',
 'tracks_is_local',
 'tracks_is_playable',
 'tracks_name',
 'tracks_popularity',
 'tracks_preview_url',
 'tracks_track_number',
 'tracks_type',
 'tracks_uri',
 'tracks_album_album_type',
 'tracks_album_href',
 'tracks_album_id',
 'tracks_album_is_playable',
 'tracks_album_name',
 'tracks_album_release_date',
 'tracks_album_release_date_precision',
 'tracks_album_total_tracks',
 'tracks_album_type',
 'tracks_album_uri',
 'tracks_artists_href',
 'tracks_artists_id',
 'tracks_artists_name',
 'tracks_artists_type',
 'tracks_artists_uri',
 'tracks_external_ids_isrc',
 'tracks_external_urls_spotify',
 'tracks_album_artists_href',
 'tracks_album_artists_id',
 'tracks_album_artists_name',
 'tracks_album_artists_type',
 'tracks_album_artists_uri',
 'tracks_album_external_urls_spotify',
 'tracks_album_images_height',
 'tracks_album_images_url',
 'tracks_album_images_width',
 'tracks_artists_external_u

In [21]:
# we don't want all of the coulmns in our silver table (like urls, href, image array columns)
df_silver = df_flatten.select('tracks_duration_ms', 'tracks_explicit', 'tracks_id', 'tracks_name', 'tracks_popularity',
 'tracks_album_id',  'tracks_album_name', 'tracks_album_artists_name', 'tracks_album_artists_id',
 'tracks_artists_id','tracks_artists_name')

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 23, Finished, Available)

In [23]:
# remove duplicate values
df_silver = df_silver.dropDuplicates()

display(df_silver)

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, 70cedeea-d0d1-463e-8440-67fddb463e42)

In [24]:
delta_path = 'Tables/raye_popular_tracks'

df_silver.write.format('delta').save(delta_path)

StatementMeta(, b274654e-c6cf-432e-9768-f8e5cca3c136, 26, Finished, Available)